In [7]:
%matplotlib inline
from pycbc.waveform import get_td_waveform
from pycbc.waveform import get_fd_waveform
from pycbc.filter import match, overlap
from pycbc.psd import aLIGOZeroDetHighPower
import pylab
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pycbc
import h5py
import sys

start = datetime.now()
print ("Start time: %s" % start)

"""
GenTemplateTD
Input: mass1 float
       mass2 float
       apx string (FD template name)
       eccentricity float (default to naught)
       long_asc_nodes float (default to naught)
       inclination float (default to naught)
       f_low float
       sample_rate int
Output: hp (time domain)
"""
def GenTemplateTD (mass1, mass2, apx, eccentricity = .0, long_asc_nodes = .0, inclination = .0, f_low=30., sample_rate=4096):
    hptilde,hctilde = get_fd_waveform(approximant=apx,
                           mass1=mass1,
                           mass2=mass2,
                           eccentricity = eccentricity,
                           long_asc_nodes = long_asc_nodes,
                           inclination = inclination,
                           f_lower=f_low,
                           delta_f=1.0/4)
    
    # FFT it to the time-domain
    tlen = int(1.0 / (1.0/sample_rate) / hptilde.delta_f)
    hptilde.resize(tlen/2 + 1)
    hp = pycbc.types.TimeSeries(pycbc.types.zeros(tlen), delta_t=1.0/sample_rate)
    pycbc.fft.ifft(hptilde, hp)
    return hp

# Generate injections
# Mass range: 1.1-1.6
# f_low = 30 (default)
# apx = "TaylorF2" (default)

masses2 = masses1 = np.linspace(1.1, 1.6, num=10)
small_inj = [None]*len(masses2)
injection_p = [small_inj]*len(masses1)

for i in range(0,len(masses1)):
    for j in range(0,len(masses2)):
        injection_p [i][j] = GenTemplateTD(masses1[i],masses2[j], apx = "TaylorF2")
        
# Grab eccentric template bank
f_ecc = h5py.File('./ebank.hdf','r')
ecc_temp_mass1 = f_ecc['mass1'][:]
ecc_temp_mass2 = f_ecc['mass2'][:]
ecc_temp_eccentricity = f_ecc['eccentricity'][:]
ecc_temp_lan = f_ecc['long_asc_nodes'][:]
ecc_temp_inc = f_ecc['inclination'][:]

ecc_temp_length = len(ecc_temp_mass1)

#ecc_temp_apx = f_ecc['approximant'][:]
ecc_temp_apx = np.empty(len(ecc_temp_mass1), object)
for i_apx in range(0,len(ecc_temp_apx)):
    ecc_temp_apx[i_apx] = 'EccentricFD'

    


# Grab non-eccentric template bank, COMMENT OUT FOR NOW
"""
f_necc = h5py.File('./stand.hdf', 'r')
necc_mass1 = f_necc['mass1'][:]
necc_mass2 = f_necc['mass2'][:]
necc_apx = f_necc['approximant'][:]
if ((len(necc_mass1) != len(necc_mass2)) or (len(necc_mass1) != len(necc_apx))):
    print ("Lengths of elements in the template bank do not match!") 
    sys.exit()
necc_temps = np.empty(len(necc_mass1))
for i in range(0,len(necc_mass1)):
    necc_temps = GenTemplateTD(mass1 = necc_mass1[i], mass2 = necc_mass2[i], apx = necc_apx[i])
"""
    

"""
GetMatch 
Input: template0 injection waveform (restricted to one polarization)
       template1 template waveform (restricted to one polarization)
       psd_file string (default to "H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt")
       f_low float
Output: match between these two waveforms, float
"""

def GetMatch (template0,template1, psd_file = "./H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt", f_low=30.):
    #resize two templates
    tlen = max(len(template0),len(template1))
    template1.resize(tlen)
    template0=template0.copy()
    template0.resize(tlen)
    
    #grab and use the psd file
    df = 1.0/template1.duration
    flen = tlen/2 +1
    my_psd = pycbc.psd.read.from_txt(filename = psd_file,
                                    length = flen,
                                    delta_f = df,
                                    low_freq_cutoff = f_low,
                                    is_asd_file = False)

    #calculate match
    m,i = match(template0,template1,psd=my_psd,low_frequency_cutoff=f_low)
    return m

"""
GetGlobalMatch
Input: i_waveform0, j_waveform0 int double index of the injection waveform (restricted to one polarization)
       tp_apx list of strs
       tp_m1, tp_m2, tp_ecc, tp_lan, temp_inc np.array of float 
       searching_radius float (only iterate through templates within a closed ball of mchirp of the injection; 
                               given as a percentage of mchirp of the injection)
       f_low float threshold frequency
Output: maximum match in the bank for the given injection waveform0, float
"""

def GetGlobalMatch(i_waveform0, j_waveform0, tp_apx, tp_m1, tp_m2, tp_ecc, tp_lan, tp_inc, searching_radius, f_low=30.):
    waveform0_m1 = masses1[i_waveform0]
    waveform0_m2 = masses2[j_waveform0]
    waveform0_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(waveform0_m1, waveform0_m2) #compute mchirp of waveform0
    waveform0 = injection_p[i_waveform0][j_waveform0]
    local_matches = np.zeros(len(tp_m1)) 
    #iterate through the given template bank
    for k in range(0,len(tp_m1)):
        this_tp_m1 = tp_m1[k]
        this_tp_m2 = tp_m2[k]
        this_tp_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(this_tp_m1, this_tp_m2) #compute mchirp of one template
        diff_mchirp = abs(waveform0_mchirp-this_tp_mchirp)
        prozent_diff = diff_mchirp/waveform0_mchirp
        if (prozent_diff > searching_radius):
            local_matches[k] = 0.
        else:
            # generate templates as needed
            one_relevant_temp = GenTemplateTD(mass1 = this_tp_m1, mass2 = this_tp_m2, apx = tp_apx[k], 
                                                   eccentricity = tp_ecc[k], long_asc_nodes = tp_lan[k],
                                                   inclination = tp_inc[k])
            local_matches[k] = GetMatch(waveform0, one_relevant_temp)
    global_match = np.amax(local_matches)
    return global_match


global_matches = np.empty([len(masses1), len(masses2)])
inj_counter = 0
# iterate through all injections
for i in range(0,len(masses1)):
    if i == 5:
        first_qtr_runtime = datetime.now()-start
        print ("Halfway through! It took us %s" % first_qtr_runtime)
    if i == 10:
        snd_runtime = datetime.now()-start
        print ("Almost through! It took us %s" % snd_runtime)
    if i == 15:
        halb_runtime = datetime.now()-start
        print ("Half way through! It took us %s" % halb_runtime)
    if i == 20:
        third_runtime = datetime.now()-start
        print ("2/3 through! It took us %s" % third_runtime)
    if i ==25:
        fourth_runtime = datetime.now()-start
        print ("5/6 through! It took us %s" % fourth_runtime)
    for j in range(0,len(masses2)):
        inj_counter += 1
        this_global_match = GetGlobalMatch(i_waveform0 = i, 
                                           j_waveform0 = j, 
                                           tp_apx = ecc_temp_apx, 
                                           tp_m1 = ecc_temp_mass1,
                                           tp_m2 = ecc_temp_mass2,
                                           tp_ecc = ecc_temp_eccentricity,
                                           tp_lan = ecc_temp_lan,
                                           tp_inc = ecc_temp_inc,
                                           searching_radius = 0.01)
        global_matches[i][j] = this_global_match
        print ("Injection number: %s" % inj_counter)
        print ("Fitting factor: %s" % this_global_match)

#coutour plot
plt.figure()
cp = plt.contourf(masses1,masses2,global_matches)
plt.colorbar(cp)
plt.xlabel('mass 1 (solar mass)')
plt.ylabel('mass 2 (solar mass)')
plt.title('Fitting factor contour plot for 10*10 injections with the eccentric template bank')
plt.show()
end=datetime.now()
total_runtime = end-start
print ("Ending time: %s" % end)
print ("Runtime: %s" % total_runtime)



        




Start time: 2019-06-27 00:02:49.738327


KeyboardInterrupt: 

In [14]:
"""
New attempt: only using FrequencySeries
"""

%matplotlib inline
from pycbc.waveform import get_fd_waveform
from pycbc.filter.matchedfilter import match
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import pycbc
import h5py
import sys

sys.stdout = open('myoutputnew.txt', 'w')



start = datetime.now()
print ("Start time: %s" % start)
sys.stdout.flush()



def GenTemplate (mass1, mass2, apx, eccentricity = .0, long_asc_nodes = .0, inclination = .0, f_low=30., freq_step=4):
    hptilde,hctilde = get_fd_waveform(approximant=apx,
                           mass1=mass1,
                           mass2=mass2,
                           eccentricity = eccentricity,
                           long_asc_nodes = long_asc_nodes,
                           inclination = inclination,
                           f_lower=f_low,
                           delta_f=1.0/freq_step)
    return hptilde

# Generate injections
# Mass range: 1.1-1.6
# f_low = 30 (default)
# apx = "TaylorF2" (default)

masses2 = masses1 = np.linspace(1.1, 1.6, num=10)
small_inj = [None]*len(masses2)
injection_p = [small_inj]*len(masses1)

for i in range(0,len(masses1)):
    for j in range(0,len(masses2)):
        injection_p [i][j] = GenTemplate(masses1[i],masses2[j], apx = "TaylorF2")
        
gen_inj_time = datetime.now()
gen_inj_duration = gen_inj_time - start
print ("Finished generating 100 injections. Duration: %s" % gen_inj_duration)
sys.stdout.flush()

        
        
# Grab eccentric template bank
f_ecc = h5py.File('./ebank.hdf','r')
ecc_temp_mass1 = f_ecc['mass1'][:]
ecc_temp_mass2 = f_ecc['mass2'][:]
ecc_temp_eccentricity = f_ecc['eccentricity'][:]
ecc_temp_lan = f_ecc['long_asc_nodes'][:]
ecc_temp_inc = f_ecc['inclination'][:]


#ecc_temp_apx = f_ecc['approximant'][:]
ecc_temp_apx = np.empty(len(ecc_temp_mass1), object)
for i_apx in range(0,len(ecc_temp_apx)):
    ecc_temp_apx[i_apx] = 'EccentricFD'

    


# Grab non-eccentric template bank, COMMENT OUT FOR NOW
"""
f_necc = h5py.File('./stand.hdf', 'r')
necc_mass1 = f_necc['mass1'][:]
necc_mass2 = f_necc['mass2'][:]
necc_apx = f_necc['approximant'][:]
if ((len(necc_mass1) != len(necc_mass2)) or (len(necc_mass1) != len(necc_apx))):
    print ("Lengths of elements in the template bank do not match!") 
    sys.exit()
necc_temps = np.empty(len(necc_mass1))
for i in range(0,len(necc_mass1)):
    necc_temps = GenTemplateTD(mass1 = necc_mass1[i], mass2 = necc_mass2[i], apx = necc_apx[i])
"""
    

"""
GetMatch 
Input: template0 injection waveform (restricted to one polarization)
       template1 template waveform (restricted to one polarization)
       psd_file string (default to "H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt")
       f_low float
Output: match between these two waveforms, float
"""

def GetMatch (template0,template1, psd_file = "./H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt", f_low=30.):
    #resize two templates
    flen = max(len(template0),len(template1))
    template1=template1.copy()
    template1.resize(flen)
    template0=template0.copy()
    template0.resize(flen)
    
    #grab and use the psd file
    df = 1.0/template1.duration
    my_psd = pycbc.psd.read.from_txt(filename = psd_file,
                                    length = flen,
                                    delta_f = df,
                                    low_freq_cutoff = f_low,
                                    is_asd_file = False)

    #calculate match
    m,i = match(template0,template1,psd=my_psd,low_frequency_cutoff=f_low)
    return m

"""
GetGlobalMatch
Input: i_waveform0, j_waveform0 int double index of the injection waveform (restricted to one polarization)
       tp_apx list of strs
       tp_m1, tp_m2, tp_ecc, tp_lan, temp_inc np.array of float 
       searching_radius float (only iterate through templates within a closed ball of mchirp of the injection; 
                               given as a percentage of mchirp of the injection)
       f_low float threshold frequency
Output: maximum match in the bank for the given injection waveform0, float
"""

def GetGlobalMatch(i_waveform0, j_waveform0, tp_apx, tp_m1, tp_m2, tp_ecc, tp_lan, tp_inc, searching_radius, f_low=30.):
    waveform0_m1 = masses1[i_waveform0]
    waveform0_m2 = masses2[j_waveform0]
    waveform0_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(waveform0_m1, waveform0_m2) #compute mchirp of waveform0
    waveform0 = injection_p[i_waveform0][j_waveform0]
    local_matches = np.zeros(len(tp_m1)) 
    #iterate through the given template bank
    for k in range(0,len(tp_m1)):
        this_tp_m1 = tp_m1[k]
        this_tp_m2 = tp_m2[k]
        this_tp_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(this_tp_m1, this_tp_m2) #compute mchirp of one template
        diff_mchirp = abs(waveform0_mchirp-this_tp_mchirp)
        prozent_diff = diff_mchirp/waveform0_mchirp
        if (prozent_diff > searching_radius):
            local_matches[k] = 0.
        else:
            # generate templates as needed
            one_relevant_temp = GenTemplate(mass1 = this_tp_m1, mass2 = this_tp_m2, apx = tp_apx[k], 
                                                   eccentricity = tp_ecc[k], long_asc_nodes = tp_lan[k],
                                                   inclination = tp_inc[k])
            local_matches[k] = GetMatch(waveform0, one_relevant_temp)
            print (local_matches[k])
            sys.stdout.flush()
    global_match = np.amax(local_matches)
    return global_match


global_matches = np.empty([len(masses1), len(masses2)])
inj_counter = 0
# iterate through all injections
for i in range(0,len(masses1)):
    if i == 5:
        first_qtr_runtime = datetime.now()-start
        print ("Halfway through! It took us %s" % first_qtr_runtime)
        sys.stdout.flush()
    if i == 10:
        snd_runtime = datetime.now()-start
        print ("Almost through! It took us %s" % snd_runtime)
        sys.stdout.flush()
    for j in range(0,len(masses2)):
        inj_counter += 1
        print ("Injection number: %s" % inj_counter)
        sys.stdout.flush()
        this_global_match = GetGlobalMatch(i_waveform0 = i, 
                                           j_waveform0 = j, 
                                           tp_apx = ecc_temp_apx, 
                                           tp_m1 = ecc_temp_mass1,
                                           tp_m2 = ecc_temp_mass2,
                                           tp_ecc = ecc_temp_eccentricity,
                                           tp_lan = ecc_temp_lan,
                                           tp_inc = ecc_temp_inc,
                                           searching_radius = 0.01)
        global_matches[i][j] = this_global_match
        print ("Fitting factor: %s" % this_global_match)
        sys.stdout.flush()

#coutour plot
plt.figure()
cp = plt.contourf(masses1,masses2,global_matches)
plt.colorbar(cp)
plt.xlabel('mass 1 (solar mass)')
plt.ylabel('mass 2 (solar mass)')
plt.title('Fitting factor contour plot for 10*10 injections with the eccentric template bank')
plt.show()
end=datetime.now()
total_runtime = end-start
print ("Ending time: %s" % end)
sys.stdout.flush()
print ("Runtime: %s" % total_runtime)
sys.stdout.flush()




KeyboardInterrupt: 